# Data Preparation for Fine-tuning

In [1]:
!pip install -U datasets


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from datasets import load_dataset

ds = load_dataset("virattt/financial-qa-10K", split="train")
ds

README.md:   0%|          | 0.00/419 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'context', 'ticker', 'filing'],
    num_rows: 7000
})

## 1. Data for Fine-tuning

Construct the dataset to the following format:

``` python
{"query": str, "pos": List[str], "neg":List[str], "pos_scores": List[int], "neg_scores": List[int], "prompt": str, "type": str}
```

`query` is the query, and `pos` is a list of positive texts, `neg` is a list of negative texts. `pos_scores` is a list of scores corresponding to the query and pos, `neg_scores` is a list of scores corresponding to the `query` and `neg` (OPTIONAL). `prompt` is the prompt used for the query, it will cover query_instruction_for_retrieval. 

If  no negative texts for a query, you can random sample some from the entire corpus as the negatives.

In [3]:
ds = ds.select_columns(column_names=["question", "context"])
ds = ds.rename_column("question", "query")
ds = ds.rename_column("context", "pos")
ds = ds.add_column("id", [str(i) for i in range(len(ds))])
ds[0]

{'query': 'What area did NVIDIA initially focus on before expanding to other computationally intensive fields?',
 'pos': 'Since our original focus on PC graphics, we have expanded to several other large and important computationally intensive fields.',
 'id': '0'}

Negative examples are important during the training of embedding models. Our initial dataset does not come with negative texts. Thus we directly sample a few from the whole corpus.

In [4]:
import numpy as np

np.random.seed(520)
neg_num = 10

def str_to_lst(data):
    data["pos"] = [data["pos"]]
    return data

# sample negative texts
new_col = []
for i in range(len(ds)):
    ids = np.random.randint(0, len(ds), size=neg_num)
    while i in ids:
        ids = np.random.randint(0, len(ds), size=neg_num)
    neg = [ds[i.item()]["pos"] for i in ids]
    new_col.append(neg)
ds = ds.add_column("neg", new_col)

# change the key of 'pos' to a list
ds = ds.map(str_to_lst)

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

In [5]:
instruction = "Represent this sentence for searching relevant passages: "
ds = ds.add_column("prompt", [instruction]*len(ds))

Now a single row of the dataset is:

In [6]:
ds[0]

{'query': 'What area did NVIDIA initially focus on before expanding to other computationally intensive fields?',
 'pos': ['Since our original focus on PC graphics, we have expanded to several other large and important computationally intensive fields.'],
 'id': '0',
 'neg': ['Kroger expects that its value creation model will deliver total shareholder return within a target range of 8% to 11% over time.',
  'CSB purchased First Mortgages of $2.9 billion during 2023.',
  'See Note 13 to our Consolidated Financial Statements for information on certain legal proceedings for which there are contingencies.',
  'Diluted earnings per share were $16.69 in fiscal 2022 compared to $15.53 in fiscal 2021.',
  'In the year ended December 31, 2023, Total net sales and revenue increased primarily due to: (1) increased net wholesale volumes primarily due to increased sales of crossover vehicles and full-size pickup trucks, partially offset by decreased sales of mid-size pickup trucks; (2) favorable Pri

Then we split the dataset into training set and testing set.

In [7]:
split = ds.train_test_split(test_size=0.1, shuffle=True, seed=520)
train = split["train"]
test = split["test"]

In [8]:
train[0]

{'query': 'What was the percentage decrease in rental costs for the U.S. markets from 2022 to 2023?',
 'pos': ['The rental costs for the U.S. markets decreased by 2.3% from 2022 to 2023, with values changing from $666.5 to $651.5.'],
 'id': '2168',
 'neg': ['In 2002, the company filed a lawsuit in Georgia seeking a declaratory judgment that it has no obligation to Aqua-Chem for any past, present, or future liabilities or expenses in connection with any claims or lawsuits against Aqua-Chem.',
  'Chevron Technical Center conducts research, develops and qualifies technology, and provides technical services and competency development in support of business outcomes. Areas of expertise include earth sciences, reservoir and production engineering, facilities engineering, reserve governance and reporting, capital projects, drilling and completions, technology ventures, and downstream technology and services.',
  'While our out-of-parathy network payment rates are on average higher than in-net

Now we are ready to store the data for later fine-tuning:

In [9]:
train.to_json("ft_data/training.json")

Creating json from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

16583481

## 2. Test Data for Evaluation

The last step is to construct the testing dataset for evaluaton.

In [10]:
test

Dataset({
    features: ['query', 'pos', 'id', 'neg', 'prompt'],
    num_rows: 700
})

First select the columns for queries:

In [14]:
queries = test.select_columns(column_names=["id", "query"])
queries = queries.rename_column("query", "text")
queries[0]

{'id': '1289',
 'text': 'How does Starbucks recognize the interest and penalties related to income tax matters on their financial statements?'}

Then select the columns for corpus:

In [15]:
corpus = ds.select_columns(column_names=["id", "pos"])
corpus = corpus.rename_column("pos", "text")

Finally, make the qrels that indicating the relations of queries and corresponding corpus"

In [16]:
qrels = test.select_columns(["id"])
qrels = qrels.rename_column("id", "qid")
qrels = qrels.add_column("docid", list(test["id"]))
qrels = qrels.add_column("relevance", [1]*len(test))
qrels[0]

{'qid': '1289', 'docid': '1289', 'relevance': 1}

Store the training set

In [17]:
queries.to_json("ft_data/test_queries.jsonl")
corpus.to_json("ft_data/corpus.jsonl")
qrels.to_json("ft_data/test_qrels.jsonl")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

30574